In [1]:
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 22.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.5 MB/s eta 0:00:00a 0:00:01


In [2]:
#%%
from Modele_decodeur_maison import *

from Pipeline import *
from Trainer import * 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 50

# Images
# images = np.load("C:/Users/lucas/Desktop/train-resnet50-res4frelu.npy")

# Texts
tokenized_fr,tokenized_en, vocab_fr,vocab_en = get_train_data_nouveau(batch_size)
n_token_fr = len(vocab_fr.keys())
n_token_en = len(vocab_en.keys())

inv_map_en = {v: k for k, v in vocab_en.items()}
inv_map_fr = {v: k for k, v in vocab_fr.items()}

n_head = 4 
num_encoder_layers = 2
num_decoder_layers = 2
dim_feedforward = 196
dropout = 0.1
activation = nn.Softmax
embedding_dim = 196

model_fr = Modèle(n_token_fr,embedding_dim,n_head, num_encoder_layers,num_decoder_layers,dim_feedforward,dropout,activation).to(device)

model_en = Modèle(n_token_en,embedding_dim,n_head, num_encoder_layers,num_decoder_layers,dim_feedforward,dropout,activation).to(device)


In [3]:
#%%
def tensor_to_sentence(output,inv_dic):
    result = [inv_dic[int(x)] for x in output]
    sentence = ""
    for word in result : 
        if word == "DEBUT_DE_PHRASE" :
            pass
        elif '@@' in word: 
            sentence+=word[:-2]
        elif word == "FIN_DE_PHRASE" :
            break 
        else :
            sentence+=word +" "
    return sentence

def traduit(model_A,model_B,data, inv_map):
    model_en.eval()
    model_fr.eval()
    output = torch.argmax(cycle_consistent_forward(model_A,model_B,data),dim = 2).T[j].T
    return tensor_to_sentence(output.view(-1),inv_map)

i = np.random.randint(len(tokenized_en)//50)
j = np.random.randint(50)
data,target = get_batch(tokenized_en, i)

# print("Phrase à traduire : \n" + tensor_to_sentence(target.T[j],inv_map_en)+ "\n Phrase traduite : \n "+ traduit(model_en,model_fr,data, inv_map_fr))


#%%

In [23]:
import os

import s3fs
import pandas
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

val_features  = np.load(fs.open('tordjx/val-resnet50-res4frelu.npy'))
train_features  = np.load(fs.open('tordjx/train-resnet50-res4frelu.npy'))


UnpicklingError: STACK_GLOBAL requires str

In [5]:
data_text, _  = get_batch(tokenized_fr, 0)
# val_features = np.load("C:/Users/lucas/Desktop/val-resnet50-res4frelu.npy")
# val_features = np.load("C:/Users/lucas/Desktop/val-resnet50-res4frelu.npy")
batchsz = 50
embed = model_fr.embedding(data_text)
# torch.cat((embed, torch.from_numpy(val_features.reshape(1014,1024,196)[:1000].reshape(1000//batchsz ,batchsz,1024 , 196)[0]).to(device))).shape

In [6]:
print(embed.shape)
print(torch.from_numpy(val_features.reshape(1014,1024,196)[:1000].reshape(1000//batchsz ,batchsz,1024 , 196)).to(device= device, dtype = torch.float32).shape)

torch.Size([50, 97, 196])
torch.Size([20, 50, 1024, 196])


In [7]:
output = model_fr(data_text, True, torch.from_numpy(val_features.reshape(1014,1024,196)[:1000].reshape(1000//batchsz ,batchsz,1024 , 196)[0]).to(device= device, dtype = torch.float32)).shape
output

torch.Size([50, 97, 196])

In [19]:
train_features= torch.from_numpy(train_features.reshape(train_features.shape[0]//batchsz ,batchsz,1024 , 196)).to(device= device, dtype = torch.float32)
train_data_fr = [tokenized_fr, train_features]
train_data_en = [tokenized_en, traun_features]
# Y=train_auto_encoding(model_fr,tokenized_fr)
# Y=train_auto_encoding(model_en,tokenized_en)
# Y= cycle_consistency_train(model_fr,model_en,tokenized_fr,tokenized_en)
mixed_train(model_fr,model_en,train_data_fr,train_data_en,200,50, True)



OutOfMemoryError: CUDA out of memory. Tried to allocate 21.68 GiB (GPU 0; 14.76 GiB total capacity; 85.88 MiB already allocated; 13.90 GiB free; 98.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [22]:
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)